Import necessary files


In [29]:
import pandas as pd
import numpy as np
from functions import *

Extraction Layer

In [30]:
# import the csv file
sales_df = pd.read_csv(r'dataset\rawdata\Sales Dataset.csv')

In [31]:
sales_df.info()
sales_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1194 entries, 0 to 1193
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      1194 non-null   object
 1   Amount        1194 non-null   int64 
 2   Profit        1194 non-null   int64 
 3   Quantity      1194 non-null   int64 
 4   Category      1194 non-null   object
 5   Sub-Category  1194 non-null   object
 6   PaymentMode   1194 non-null   object
 7   Order Date    1194 non-null   object
 8   CustomerName  1194 non-null   object
 9   State         1194 non-null   object
 10  City          1194 non-null   object
 11  Year-Month    1194 non-null   object
dtypes: int64(3), object(9)
memory usage: 70.0+ KB


,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,Order Date,CustomerName,State,City,Year-Month
0,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06
1,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12
2,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2021-07-25,Robert Stone,New York,Buffalo,2021-07
3,B-26776,4975,1330,14,Electronics,Printers,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06
4,B-26776,4975,1330,14,Electronics,Printers,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12


### Handle missing Values

In [32]:
# check for null
sales_df.isnull()


,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,Order Date,CustomerName,State,City,Year-Month
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1189,False,False,False,False,False,False,False,False,False,False,False,False
1190,False,False,False,False,False,False,False,False,False,False,False,False
1191,False,False,False,False,False,False,False,False,False,False,False,False
1192,False,False,False,False,False,False,False,False,False,False,False,False


In [33]:
sales_df.columns

Index(['Order ID', 'Amount', 'Profit', 'Quantity', 'Category', 'Sub-Category',
       'PaymentMode', 'Order Date', 'CustomerName', 'State', 'City',
       'Year-Month'],
      dtype='object')

In [34]:
#standardize column names
sales_df = standardize_column_names(sales_df)

sales_df.columns

Index(['order_id', 'amount', 'profit', 'quantity', 'category', 'sub_category',
       'paymentmode', 'order_date', 'customername', 'state', 'city',
       'year_month'],
      dtype='object')

In [35]:
# change date column to date data type
sales_df["order_date"] = pd.to_datetime(sales_df["order_date"])
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1194 entries, 0 to 1193
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      1194 non-null   object        
 1   amount        1194 non-null   int64         
 2   profit        1194 non-null   int64         
 3   quantity      1194 non-null   int64         
 4   category      1194 non-null   object        
 5   sub_category  1194 non-null   object        
 6   paymentmode   1194 non-null   object        
 7   order_date    1194 non-null   datetime64[ns]
 8   customername  1194 non-null   object        
 9   state         1194 non-null   object        
 10  city          1194 non-null   object        
 11  year_month    1194 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 74.7+ KB


In [36]:
# remove duplicate 
print(f"total record {len(sales_df)}")
sales_df = sales_df.drop_duplicates()
print(f"total record {len(sales_df)}")

total record 1194
total record 1194


In [37]:
# rename order id column
sales_df = sales_df.rename(columns={'order_id': 'order_no'})
sales_df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1194 entries, 0 to 1193
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_no      1194 non-null   object        
 1   amount        1194 non-null   int64         
 2   profit        1194 non-null   int64         
 3   quantity      1194 non-null   int64         
 4   category      1194 non-null   object        
 5   sub_category  1194 non-null   object        
 6   paymentmode   1194 non-null   object        
 7   order_date    1194 non-null   datetime64[ns]
 8   customername  1194 non-null   object        
 9   state         1194 non-null   object        
 10  city          1194 non-null   object        
 11  year_month    1194 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 84.0+ KB


In [38]:
# create a new order id column 
sales_df = sales_df.reset_index(drop=True)
sales_df['order_id'] = range(1, len(sales_df) + 1)

In [39]:
sales_df.info()
sales_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1194 entries, 0 to 1193
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_no      1194 non-null   object        
 1   amount        1194 non-null   int64         
 2   profit        1194 non-null   int64         
 3   quantity      1194 non-null   int64         
 4   category      1194 non-null   object        
 5   sub_category  1194 non-null   object        
 6   paymentmode   1194 non-null   object        
 7   order_date    1194 non-null   datetime64[ns]
 8   customername  1194 non-null   object        
 9   state         1194 non-null   object        
 10  city          1194 non-null   object        
 11  year_month    1194 non-null   object        
 12  order_id      1194 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(8)
memory usage: 79.4+ KB


,order_no,amount,profit,quantity,category,sub_category,paymentmode,order_date,customername,state,city,year_month,order_id
0,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06,1
1,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12,2
2,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2021-07-25,Robert Stone,New York,Buffalo,2021-07,3
3,B-26776,4975,1330,14,Electronics,Printers,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06,4
4,B-26776,4975,1330,14,Electronics,Printers,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12,5


In [40]:
sales_df.columns

Index(['order_no', 'amount', 'profit', 'quantity', 'category', 'sub_category',
       'paymentmode', 'order_date', 'customername', 'state', 'city',
       'year_month', 'order_id'],
      dtype='object')

In [54]:
# Nomarlization
#customer 
customer_df = sales_df[['customername','city', 'state']].drop_duplicates().reset_index(drop=True)
customer_df['customer_id'] = range(1, len(customer_df) + 1)
customer_df.head()


,customername,city,state,customer_id
0,David Padilla,Miami,Florida,1
1,Connor Morgan,Chicago,Illinois,2
2,Robert Stone,Buffalo,New York,3
3,John Fields,Orlando,Florida,4
4,Clayton Smith,Miami,Florida,5


In [56]:
product_df = sales_df[['category', 'sub_category']].drop_duplicates().reset_index(drop=True)
product_df['product_id'] = range(1, len(product_df) + 1)
product_df.head()

,category,sub_category,product_id
0,Electronics,Electronic Games,1
1,Electronics,Printers,2
2,Office Supplies,Pens,3
3,Electronics,Laptops,4
4,Furniture,Tables,5


In [57]:
payment_df = sales_df[['paymentmode']].drop_duplicates().reset_index(drop=True)
payment_df['payment_id'] = range(1, len(payment_df) + 1)
payment_df.head()

,paymentmode,payment_id
0,UPI,1
1,Debit Card,2
2,EMI,3
3,Credit Card,4
4,COD,5


In [58]:
location_df = sales_df[['city','state']].copy().drop_duplicates().reset_index(drop=True)
location_df['location_id'] = range(1, len(location_df) + 1)
location_df.head()

,city,state,location_id
0,Miami,Florida,1
1,Chicago,Illinois,2
2,Buffalo,New York,3
3,Orlando,Florida,4
4,Los Angeles,California,5


In [61]:
# Merge to build orders sales with foreign keys
order_df = sales_df.merge(customer_df, on=['customername', 'city', 'state']) \
    .merge(product_df, on=['category', 'sub_category']) \
    .merge(payment_df, on=['paymentmode']) \
    .merge(location_df, on=['city', 'state'])

# Add primary key and surrogate key
order_df = order_df.reset_index(drop=True)
order_df['order_id'] = range(1, len(order_df) + 1)
#orders_df['order_id'] = orders_df.index + 1  # PK
#orders_df['order_sk'] = orders_df['order_id']  # SK (same as PK for now)

# Final fact table (OLTP-level)
order_df = order_df[[
    'order_id',  'order_no', 'order_date', 'year_month',
    'customer_id', 'product_id', 'payment_id', 'location_id',
    'amount', 'profit', 'quantity'
]]

display(order_df.head(3))
display(customer_df.head(3))
display(product_df.head(3))
display(payment_df.head(3))
display(location_df.head(3))


,order_id,order_no,order_date,year_month,customer_id,product_id,payment_id,location_id,amount,profit,quantity
0,1,B-26776,2023-06-27,2023-06,1,1,1,1,9726,1275,5
1,2,B-25714,2025-02-19,2025-02,140,1,1,1,6962,3429,12
2,3,B-26224,2023-12-13,2023-12,262,1,1,1,3953,1776,15


,customername,city,state,customer_id
0,David Padilla,Miami,Florida,1
1,Connor Morgan,Chicago,Illinois,2
2,Robert Stone,Buffalo,New York,3


,category,sub_category,product_id
0,Electronics,Electronic Games,1
1,Electronics,Printers,2
2,Office Supplies,Pens,3


,paymentmode,payment_id
0,UPI,1
1,Debit Card,2
2,EMI,3


,city,state,location_id
0,Miami,Florida,1
1,Chicago,Illinois,2
2,Buffalo,New York,3
